## Task 1
Начнем с простого вопроса: что произойдет в результате выполнения следующей команды?

In [14]:
import pandas as pnds

In [26]:
# This will fail because pandas is not defined
# pandas.read_csv() 

# This will work
# pd.read_csv()

## Task 2
Импортируйте библиотеку pandas, а затем загрузите данные по вышеупомянутой ссылке.  

Результат сохраните в переменную taxi.

In [18]:
import pandas as pd
path_to_file = '2_taxi_nyc.csv'
taxi = pd.read_csv(path_to_file)

## Task 3
Проверьте, сколько всего строк и столбцов имеется в датасете.

In [30]:
taxi.shape

(29101, 14)

## Task 4
Давайте посмотрим на типы колонок. Все ли из них считались правильно? В качестве ответа выберите тип, преобладающий в датасете.

In [37]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29101 entries, 0 to 29100
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pickup_dt     29101 non-null  object 
 1   pickup_month  29101 non-null  object 
 2   borough       26058 non-null  object 
 3   pickups       29101 non-null  int64  
 4   hday          29101 non-null  object 
 5   spd           29101 non-null  float64
 6   vsb           29101 non-null  float64
 7   temp          29101 non-null  float64
 8   dewp          29101 non-null  float64
 9   slp           29101 non-null  float64
 10  pcp 01        29101 non-null  float64
 11  pcp 06        29101 non-null  float64
 12  pcp 24        29101 non-null  float64
 13  sd            29101 non-null  float64
dtypes: float64(9), int64(1), object(4)
memory usage: 3.1+ MB


## Task 5
В названиях столбцов pcp 01, pcp 06, pcp 24 встречается пробел. Это не очень удобно, если в дальнейшем вы планируете обращаться к колонкам через точку, без использования кавычек и скобочек. 

Замените пробел в названиях на знак нижнего подчеркивания. 

Данные должны быть изменены в этом же датафрейме.

In [21]:
taxi.rename(lambda x: x.replace(' ', '_'), axis = 'columns', inplace = True)

## Task 6
Сколько раз в данных встречается район Бруклин (Brooklyn)?


In [31]:
(
    taxi
    .borough
    .value_counts()
)

Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: borough, dtype: int64

## Task 7
Следующая задача: выяснить, из какого района было совершено наибольшее количество поездок за весь период. 
Для начала, посчитайте общее количество поездок (pickups), без группировки.


In [78]:
(
    taxi
    .pickups
    .sum()
)

14265773

## Task 8
А теперь сгруппируйте данные по району (borough) и укажите, из какого пункта было совершено больше всего поездок.


In [86]:
(
    taxi
    .groupby('borough', as_index=False)
    .agg({'pickups': 'sum'})
    .sort_values('pickups', ascending=False)
)

,borough,pickups
3,Manhattan,10367841
1,Brooklyn,2321035
4,Queens,1343528
0,Bronx,220047
5,Staten Island,6957
2,EWR,105


## Task 9
Сохраните название района с наименьшим числом поездок в переменную min_pickups, применив подходящий метод.

In [32]:
min_pickups = (
    taxi
    .groupby('borough')
    .agg({'pickups': 'sum'})
    .idxmin()
    .pickups
)
min_pickups

'EWR'

## Task 10
Продолжим изучение данных и посмотрим на число поездок в выходные дни. 

Сгруппируйте данные по двум признакам: району города и является ли день выходным (колонки borough и hday). 

Сравните среднее число поездок, и выберите районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни. 

In [47]:
(
    taxi
    .groupby(['borough', 'hday'], as_index=False)
    .agg({'pickups': 'mean'})
    .pivot(index = 'borough', columns = 'hday', values = 'pickups')
    .reset_index()
    .query('''Y > N''')
)

hday,borough,N,Y
2,EWR,0.023467,0.041916
4,Queens,308.899904,320.730539


## Task 11
Для каждого района посчитайте число поездок по месяцам. 

Отсортируйте полученные значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor.

Обратите внимание, что итоговый датасет должен состоять из 3-х колонок - pickup_month, borough, pickups.



In [110]:
pickups_by_mon_bor = (
    taxi
    .groupby(['pickup_month', 'borough'], as_index=False)
    .agg({'pickups': 'sum'})
    .sort_values('pickups', ascending=False)
)


## Task 12
Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F, и возвращает значения, переведенные в градусы Цельсия.

In [121]:
def temp_to_celcius(fahrenheit): 
    celsius = ((fahrenheit-32)*5)/9
    return celsius

In [124]:
taxi['celcius_temp'] = taxi.transform({'temp': temp_to_celcius})
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd,celcius_temp
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,-1.111111
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,-1.111111
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,-1.111111
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,-1.111111
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,-1.111111
